In [1]:
import pandas as pd
import talib as ta
import numpy as np
import plotly.graph_objs as go

import datetime as dt
from datetime import date
import pandas_datareader.data as pdr
import os

In [2]:
def get_stock_data_Internet(code) :
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [3]:
def get_stock_data_local(dirname, code) :
  dir_path = f"{dirname}"
  csv_name = f'{code}_JP.csv'
  csv_path = os.path.join(dir_path, csv_name)
  dtype_dict = {'Open': float, 'High':float, 'Low':float, 'Close':float, 'Volume':float}
  df = pd.read_csv(csv_path, index_col=0, usecols=["Date", "Open", "High", "Low", "Close", "Volume"],
                  dtype=dtype_dict)
  df.index = pd.to_datetime(df.index).astype('<M8[ns]')
  
  return df

In [4]:
#@return (boolean GC, boolean DC)
def GCDC(dirname, code, day) :
        gc_flag = False
        dc_flag = False
        #ここを変えればよい
        df = get_stock_data_local(dirname, code)
        close =df["Close"]
        #移動平均線
        df["ma5"] = ta.SMA(close, timeperiod=5)
        df['ma25'] = ta.SMA(close, timeperiod=25)

        #ゴールデンクロス、デッドクロス
        cross = df["ma5"] > df["ma25"]
        df["cross"] = cross
        cross_shift = cross.shift(1)
        temp_gc = (cross != cross_shift) & (cross ==True)
        temp_dc = (cross != cross_shift) & (cross == False)
        gc = [m if g == True else np.nan for g, m in zip(temp_gc, df['ma5'])]
        dc = [m if d == True else np.nan for d, m in zip(temp_dc, df['ma25'])]
        df["gc"], df["dc"] = gc, dc


        #------------------------日付調整-------------------------------
        today = date.today()
        few_ago_days = today - dt.timedelta(days = day)
        rdf = df[dt.datetime(few_ago_days.year, few_ago_days.month, few_ago_days.day):]
        rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

        #GCまたはDCをしている日付の抽出
        gc_index = rdf.query("gc == gc").index
        dc_index = rdf.query("dc == dc").index

        if len(gc_index) != 0 :
                gc_flag = True

        if len(dc_index) != 0 :
                dc_flag = True

        return gc_flag, dc_flag

In [7]:
#銘柄のコードとGDDCをしたかどうかを表示する
#dayは何日さかのぼるか
def GCDC_PrintAll(dirname, day) :
    dirname = "Prime_data"
    day = 5

    cn = pd.read_csv('all_stock_data.csv')
    cn = cn.drop(['33IndustryClass', '33IndustryCode', '17IndustryClass', '17IndustryCode'], axis = 1)
    if dirname == "Prime_data" :
        cn = cn[cn['MarketSegment'].isin(['プライム（内国株式）'])]
    if dirname == "Standard_data" :
        cn = cn[cn['MarketSegment'].isin(['スタンダード（内国株式）'])]
    if dirname == "Growth_data" :
        cn = cn[cn['MarketSegment'].isin(['グロース（内国株式）'])]

    codes = cn["Code"]

    cn["GC"] = np.nan
    cn["DC"] = np.nan

    for code in codes :
        target_index = cn.query(f"Code == {code}").index[0]
        print(code, end=' : ')
        if GCDC(dirname, code, day)[0] == True:
            cn.loc[[target_index],"GC"] = True
            print("GC" , end = " ")
        if GCDC(dirname, code, day)[1] == True:
            cn.loc[[target_index],"DC"] = True
            print("DC")
            continue
        else :
            print("None")

In [6]:
def GCDC_Code_List(dirname, day, gcresult, dcresult) :
    gcdclist = []
    cn = pd.read_csv('all_stock_data.csv')
    cn = cn.drop(['33IndustryClass', '33IndustryCode', '17IndustryClass', '17IndustryCode'], axis = 1)
    if dirname == "Prime_data" :
        cn = cn[cn['MarketSegment'].isin(['プライム（内国株式）'])]
    if dirname == "Standard_data" :
        cn = cn[cn['MarketSegment'].isin(['スタンダード（内国株式）'])]
    if dirname == "Growth_data" :
        cn = cn[cn['MarketSegment'].isin(['グロース（内国株式）'])]

    codes = cn["Code"]

    for code in codes :
        gcdcResult = GCDC(dirname, code, day)
        if gcdcResult[0] == gcresult & gcdcResult[1] == dcresult :
            gcdclist.append(code)   
    return gcdclist

In [9]:
gcdclist = GCDC_Code_List("Prime_data", 5, True, False)
len(gcdclist)

1834